### What is this?
For stock prices prediction, we will probably text input or sentiment input.
To get familiar with those kind of input, I think it's good to learn them by developing Chatbot.
It's also fun :)

### Steps
- Input data: Use my own twitter data
- Shape the input data
- Build a model
- Train
- Test
- Visualize

### Input Data
I'm going to use my own twitter data.

### Tokenizer

    # Install mecab
    # Thanks to http://qiita.com/taroc/items/b9afd914432da08dafc8
    brew install mecab
    brew install mecab-ipadic
    pip install mecab-python3

In [20]:
import MeCab
mt = MeCab.Tagger("mecabrc")
mt.parse('') # prevent string gc-ed huh?
node = mt.parseToNode("最近自分はどこに行こうとしてるんだろと思いながらトレーニングしてる😉")
while node:
    word = node.surface
    pos = node.feature.split(",")[0]
    print('{0} , {1}'.format(word, pos))
    node = node.next

 , BOS/EOS
最近 , 名詞
自分 , 名詞
は , 助詞
どこ , 名詞
に , 助詞
行こ , 動詞
う , 助動詞
と , 助詞
し , 動詞
てる , 動詞
ん , 名詞
だろ , 助動詞
と , 助詞
思い , 動詞
ながら , 助詞
トレーニング , 名詞
し , 動詞
てる , 動詞
😉 , 記号
 , BOS/EOS


In [43]:
import MeCab
mt = MeCab.Tagger("mecabrc")
mt.parse('') # prevent string gc-ed huh?

def tokenize(text):
    ret = []
    node = mt.parseToNode(text)
    while node:
        word = node.surface
        if word == '':
            word = 'BOS/EOS'
        ret.append(word)
        node = node.next
    return ret

tokenize("最近自分はどこに行こうとしてるんだろと思いながらトレーニングしてる😉")

['BOS/EOS',
 '最近',
 '自分',
 'は',
 'どこ',
 'に',
 '行こ',
 'う',
 'と',
 'し',
 'てる',
 'ん',
 'だろ',
 'と',
 '思い',
 'ながら',
 'トレーニング',
 'し',
 'てる',
 '😉',
 'BOS/EOS']

### Install Seq2Seq
See http://qiita.com/halhorn/items/dc10596942ef4be54af5

    git clone https://www.github.com/datalogai/recurrentshop.git
    cd recurrentshop
    python setup.py install
    pip3 install git+https://github.com/farizrahman4u/seq2seq.git

In [22]:
from seq2seq.models import SimpleSeq2Seq

In [23]:
model = SimpleSeq2Seq(input_dim=1, hidden_dim=10, output_length=8, output_dim=1)
print(model)

In [32]:
import random as random
import string as string

def randomString(n):
    return ''.join([random.choice(string.ascii_letters + string.digits) for i in range(n)])

print(randomString(10))

cMhNVE7msy


In [36]:
## Create Input data
num_input = 50
print([randomString(5) for _ in range(num_input)])

## never mind, this is not necesary for now

['g7EAd', 'przaF', 'UjkGS', 'ymZxw', 'Lc2E9', 'XbzpE', 'Er1Rc', 'moe90', 'em9SN', '6T11D', 'd9DGY', 'KRlXR', '3Z6iK', 'cFJQ9', 'PhcRD', 'ubm02', 'bQWgD', 'k536R', 'isOkm', 'XC2lM', 'Ct8Gp', 'FNZKB', 'R4bJK', 'R2ljI', 'd2VAx', 'sUyDB', '5QJpn', 'FzLDC', 'XVIEC', 'luv6W', 'WEm8Q', 'qGcDl', '6dmqr', '5UZwT', 'Y4Y2V', 'fex2t', 'NVO6c', 'AzVVn', 'SZris', 'EVL6i', 'QAt5e', 'eURi5', 'WOmFd', 'EGi5t', 'cIMfe', 'vWmmn', 'c4UpQ', 'MQ3dX', '7vnmg', '58eR3']


## Steps
- Understand http://qiita.com/halhorn/items/dc10596942ef4be54af5
- Make above work here
- Change above a little bit
- Use 10 pairs of replies and train then predict
- Run above via real data
- Can we use word2vec above?

In [101]:
## from http://qiita.com/halhorn/items/dc10596942ef4be54af5
from seq2seq.models import SimpleSeq2Seq
import numpy as np
import matplotlib.pylab as plt

# シンプルな Seq2Seq モデルを構築
model = SimpleSeq2Seq(input_dim=1, hidden_dim=10, output_length=8, output_dim=1)

# 学習の設定
model.compile(loss='mse', optimizer='rmsprop')

# データ作成
# 入力：1000パターンの位相を持つ一次元のサイン波
# 出力：各入力の逆位相のサイン波
a = np.random.random(1000)
x = np.array([np.sin([[p] for p in np.arange(0, 0.8, 0.1)] + aa) for aa in a])
print(x.shape)
y = -x


(1000, 8, 1)


In [125]:

# 学習
model.fit(x, y, nb_epoch=5, batch_size=32)

# 未学習のデータでテスト
x_test = np.array([np.sin([[p] for p in np.arange(0, 0.8, 0.1)] + aa) for aa in np.arange(0, 1.0, 0.1)])
y_test = -x_test
print(model.evaluate(x_test, y_test, batch_size=32))

# 未学習のデータで生成
predicted = model.predict(x_test, batch_size=32)

plt.plot(np.arange(0, 0.8, 0.1), [xx[0] for xx in x_test[9]])
plt.plot(np.arange(0, 0.8, 0.1), [xx[0] for xx in predicted[9]])
plt.show()

ValueError: Error when checking model target: expected recurrentcontainer_16 to have shape (None, 16, 1) but got array with shape (1000, 8, 1)

In [47]:
conversations = [
    ("オンライン証券会社、ETFなどの商品の良さ、手数料の安さで考えると最近はどこがいいんだろう？２つほど口座持ってるんだけど5年前くらいに選んだものだからベストかわからず。 @karino2012 さんとかどうですか。", "最近の事は自分も分からんですねぇ。7年くらい前からE-Tradeと楽天の2つがあれば良さそうだったけど、、、"),
    ("なぜ決定木を使う人が多いんだろう。", "識別能力が高くて安定しているからじゃないか。")
]

In [114]:
tweets = conversations[0]
replies = conversations[1]

tweets_lemmatized = [tokenize(sentence) for sentence in tweets]
max_len_tweets = len(max(tweets_lemmatized, key=len))
print("max_len_tweets", max_len_tweets)

replies_lemmatized = [tokenize(sentence) for sentence in replies]
max_len_replies = len(max(replies_lemmatized, key=len))
print("max_len_replies", max_len_replies)

tweets_lemmatized = [tweet + ['BOS/EOS'] * (max_len_tweets - len(tweet)) for tweet in tweets_lemmatized]
replies_lemmatized = [reply + ['BOS/EOS'] * (max_len_replies - len(reply)) for reply in replies_lemmatized]

# pad BOS/EOS
print(tweets_lemmatized)

max_len_tweets 62
max_len_replies 16
[['BOS/EOS', 'オンライン', '証券', '会社', '、', 'ETF', 'など', 'の', '商品', 'の', '良', 'さ', '、', '手数料', 'の', '安', 'さ', 'で', '考える', 'と', '最近', 'は', 'どこ', 'が', 'いい', 'ん', 'だろ', 'う', '？', '２つ', 'ほど', '口座', '持っ', 'てる', 'ん', 'だ', 'けど', '5', '年', '前', 'くらい', 'に', '選ん', 'だ', 'もの', 'だ', 'から', 'ベスト', 'か', 'わから', 'ず', '。', '@', 'karino', '2012', 'さん', 'とか', 'どう', 'です', 'か', '。', 'BOS/EOS'], ['BOS/EOS', '最近', 'の', '事', 'は', '自分', 'も', '分から', 'ん', 'です', 'ねぇ', '。', '7', '年', 'くらい', '前', 'から', 'E', '-', 'Trade', 'と', '楽天', 'の', '2', 'つ', 'が', 'あれ', 'ば', '良', 'さ', 'そう', 'だっ', 'た', 'けど', '、', '、', '、', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS']]


In [115]:
import itertools
words = set(itertools.chain(*tweets_lemmatized)).union(set(itertools.chain(*replies_lemmatized)))
print("words", words)

words {'て', 'ベスト', '7', 'さん', 'BOS/EOS', '2', '木', 'など', 'た', 'くらい', 'の', 'も', 'に', '選ん', '決定', 'う', '良', 'で', '会社', 'そう', '、', 'ほど', 'と', '安', 'ねぇ', '考える', '証券', 'ば', 'てる', 'か', 'わから', '2012', 'Trade', '分から', 'つ', 'あれ', '口座', 'E', 'なぜ', 'どう', '識別', '5', 'ない', '安定', 'ず', 'じゃ', 'けど', '使う', 'し', '事', '２つ', 'だっ', 'オンライン', '自分', '。', '商品', 'が', '？', 'ETF', '年', '持っ', 'だ', 'いる', '-', 'どこ', '能力', '高く', '最近', '@', 'さ', '前', '手数料', 'もの', 'ん', 'から', '多い', 'とか', 'を', 'karino', 'は', 'いい', '人', 'だろ', 'です', '楽天'}


In [116]:
# dictionaries for converting words to integers and vice versa
word2idx = dict((v, i) for i, v in enumerate(words))
print("word2idx", word2idx)

idx2word = list(words)
print("idx2word", idx2word)

word2idx {'て': 0, 'ベスト': 1, '多い': 75, 'ず': 44, '7': 2, 'さん': 3, 'BOS/EOS': 4, '木': 6, 'など': 7, 'し': 48, 'た': 8, '２つ': 50, 'karino': 78, 'くらい': 9, 'の': 10, '自分': 53, 'に': 12, '選ん': 13, 'だっ': 51, 'から': 74, 'もの': 72, 'じゃ': 45, 'オンライン': 52, '決定': 14, 'う': 15, '良': 16, '。': 54, 'だ': 61, 'で': 17, 'あれ': 35, '会社': 18, '2': 5, '@': 68, '？': 57, 'そう': 19, 'ETF': 58, '年': 59, 'けど': 46, '、': 20, '持っ': 60, 'ねぇ': 24, 'と': 22, '安': 23, '人': 81, '高く': 66, 'ほど': 21, '-': 63, 'どこ': 64, '証券': 26, 'ば': 27, 'てる': 28, 'E': 37, 'か': 29, 'わから': 30, '使う': 47, '2012': 31, 'だろ': 82, 'さ': 69, '事': 49, '手数料': 71, 'ん': 73, '商品': 55, '前': 70, '能力': 65, 'です': 83, 'Trade': 32, '分から': 33, 'つ': 34, '口座': 36, 'も': 11, 'は': 79, 'なぜ': 38, 'を': 77, 'いい': 80, 'どう': 39, '最近': 67, '識別': 40, 'いる': 62, '5': 41, 'が': 56, '考える': 25, 'ない': 42, '楽天': 84, 'とか': 76, '安定': 43}
idx2word ['て', 'ベスト', '7', 'さん', 'BOS/EOS', '2', '木', 'など', 'た', 'くらい', 'の', 'も', 'に', '選ん', '決定', 'う', '良', 'で', '会社', 'そう', '、', 'ほど', 'と', '安', 'ねぇ', '考える', '

In [117]:
# convert the sentences a numpy array
to_idx = lambda x: [word2idx[word] for word in x]

tweets_idx = [to_idx(tweet) for tweet in tweets_lemmatized]
replies_idx = [to_idx(tweet) for tweet in replies_lemmatized]

print("tweets_idx", tweets_idx)
print("replies_idx", replies_idx)

tweets_idx [[4, 52, 26, 18, 20, 58, 7, 10, 55, 10, 16, 69, 20, 71, 10, 23, 69, 17, 25, 22, 67, 79, 64, 56, 80, 73, 82, 15, 57, 50, 21, 36, 60, 28, 73, 61, 46, 41, 59, 70, 9, 12, 13, 61, 72, 61, 74, 1, 29, 30, 44, 54, 68, 78, 31, 3, 76, 39, 83, 29, 54, 4], [4, 67, 10, 49, 79, 53, 11, 33, 73, 83, 24, 54, 2, 59, 9, 70, 74, 37, 63, 32, 22, 84, 10, 5, 34, 56, 35, 27, 16, 69, 19, 51, 8, 46, 20, 20, 20, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]
replies_idx [[4, 38, 14, 6, 77, 47, 81, 56, 75, 73, 82, 15, 54, 4, 4, 4], [4, 40, 65, 56, 66, 0, 43, 48, 0, 62, 74, 45, 42, 29, 54, 4]]


In [122]:
X = np.array(tweets_idx).reshape(-1, max_len_tweets, 1)
#print("X", X)
print("X.shape", X.shape)

Y = np.asarray(replies_idx).reshape(-1, max_len_replies, 1)
print("Y.shape", Y.shape)

X.shape (2, 62, 1)
Y.shape (2, 16, 1)


In [134]:
model = SimpleSeq2Seq(input_dim=1, hidden_dim=10, output_length=max_len_replies, output_dim=1, depth=3)
model.compile(loss='mse', optimizer='adam')

model.fit(X, Y, nb_epoch=5, batch_size=32)

Epoch 1/5
2/2 [==============================] - 3s - loss: 2332.6660
Epoch 2/5
2/2 [==============================] - 0s - loss: 2331.1233
Epoch 3/5
2/2 [==============================] - 0s - loss: 2329.5732
Epoch 4/5
2/2 [==============================] - 0s - loss: 2328.0342
Epoch 5/5
2/2 [==============================] - 0s - loss: 2326.5234


In [136]:
test_str="いい人だろ"
lemmatized = tokenize(test_str)
lemmatized = lemmatized + ['BOS/EOS'] * (max_len_tweets - len(lemmatized))
print(lemmatized)

idx = to_idx(lemmatized)
print(idx)
model.predict(np.array(idx).reshape(1, max_len_tweets, 1))

['BOS/EOS', 'いい', '人', 'だろ', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS', 'BOS/EOS']
[4, 80, 81, 82, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


array([[[ 0.00495657],
        [ 0.01447026],
        [ 0.02855   ],
        [ 0.046618  ],
        [ 0.06772517],
        [ 0.09072167],
        [ 0.11441334],
        [ 0.13770361],
        [ 0.1597044 ],
        [ 0.17979693],
        [ 0.1976383 ],
        [ 0.21312347],
        [ 0.22632433],
        [ 0.23742503],
        [ 0.24666676],
        [ 0.25430775]]], dtype=float32)